In [1]:
import cptac
import scipy
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statistics
import parse_correlations_dataframe as get_corr
import copy

cptac warning: Your version of cptac (0.8.5) is out-of-date. Latest is 0.8.6. Please run 'pip install --upgrade cptac' to update it. (<ipython-input-1-147bbc873c03>, line 1)


In [8]:
def find_mut_tumor(cancer_type, gene):
    gene_multi = cancer_type.multi_join({'proteomics': gene, 'transcriptomics': gene, 'somatic_mutation': gene}, tissue_type = 'tumor')

    if gene_multi[str(gene + '_proteomics')].isnull().values.any():
        return float("NaN")
    trans = list(gene_multi[str(gene +"_transcriptomics")])
    prot = list(gene_multi[str(gene + '_proteomics')])
    group = []
    for i in gene_multi[str(gene + '_Mutation_Status')]:
        if type(i) == str:
            group.append("mutation")
        else:
            group.append("wt")           
    gene_df = pd.DataFrame({'Type': group, 'Proteomics': prot, 'Transcriptomics': trans})
    return gene_df

In [2]:
cptac.download("endometrial")
en = cptac.Endometrial()

In [11]:
find_mut_tumor(en, 'ABL1')

Name        ABL1_proteomics  ABL1_transcriptomics        ABL1_Mutation  \
Patient_ID                                                               
C3L-00006           -0.2960                 11.36  [Missense_Mutation]   
C3L-00008           -0.0897                 11.77                  NaN   
C3L-00032           -0.1770                 11.04                  NaN   
C3L-00090           -0.5130                 10.93                  NaN   
C3L-00098           -0.5260                 10.57                  NaN   
...                     ...                   ...                  ...   
C3N-01520           -0.1810                 11.98                  NaN   
C3N-01521           -0.6250                 11.29                  NaN   
C3N-01537           -0.2450                 11.40                  NaN   
C3N-01802            0.1190                 10.58                  NaN   
C3N-01825            0.1480                 11.22                  NaN   

Name       ABL1_Location ABL1_Mutatio

,Type,Proteomics,Transcriptomics
0,mutation,-0.2960,11.36
1,wt,-0.0897,11.77
2,wt,-0.1770,11.04
3,wt,-0.5130,10.93
4,wt,-0.5260,10.57
...,...,...,...
90,wt,-0.1810,11.98
91,wt,-0.6250,11.29
92,wt,-0.2450,11.40
93,wt,0.1190,10.58
